# Imports

In [11]:
# تحميل المكتبات اللازمة
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
import joblib
from sklearn.metrics import r2_score



تنبؤ العقارات (تنبؤ OutcomeVariable)

فيما يلي وصف لكل عمود:

# المعلومات العامة

### OutcomeVariable: المقياس الأساسي الذي نهدف إلى التنبؤ به أو تحليله لكل عنصر في مجموعة البيانات.
### BuildingCategory: رمز رقمي يصنف النوع الهيكلي العام للعنصر.
### ZoningClassification: فئة الاستخدام المحددة للمنطقة التي يقع فيها العنصر.
### StreetLineLength: طول واجهة العنصر على الشارع بوحدات خطية.
### ParcelSize: المساحة الإجمالية للأرض المرتبطة بالعنصر، مقاسة بوحدات مربعة.
### RoadAccessType: نوع الطريق الذي يوفر الوصول إلى موقع العنصر.
### AlleyAccessType: نوع الوصول إلى الزقاق المتاح لموقع العنصر (إن وجد).
### ParcelShape: الشكل الهندسي العام لقطعة الأرض.
### TerrainFlatness: وصف لمستوى الأرض أو انحدارها.
### UtilityAvailability: أنواع خدمات المرافق المتاحة في موقع العنصر.
### ParcelSettings: تكوينات أو ميزات محددة لقطعة الأرض.
### TerrainSlope: درجة ميل الأرض.
### District: المناطق الجغرافية الفرعية أو المناطق داخل منطقة أكبر.
### RoadProximity1: يشير إلى قرب العنصر من طريق رئيسي أو مسار نقل.
### RoadProximity2: يشير إلى قرب العنصر من طريق رئيسي ثانٍ أو مسار، إن وجد.

# خصائص العنصر

### DwellingType: التصنيف المعماري المحدد للهيكل.
### DwellingStyle: النمط المعماري للهيكل.
### MaterialQuality: تصنيف الجودة العامة للمواد المستخدمة في بناء العنصر.
### ConditionRating: تقييم عام لحالة العنصر الحالية من حيث الصيانة والإصلاح.
### ConstructionYear: السنة التي بُني فيها العنصر أصلاً.
### RenovationYear: السنة التي شهد فيها العنصر تجديدًا كبيرًا أو إضافة.
### RoofType: نمط أو تصميم السقف.
### RoofMaterial: المادة الأساسية المستخدمة في بناء السقف.
### ExteriorCladding1: المادة الرئيسية التي تغطي الجزء الخارجي للعنصر.
### ExteriorCladding2: مادة تغطية خارجية ثانوية، إن وجدت.
### MasonryType: نوع العمل الزخرفي بالطوب المستخدم في الجزء الخارجي للعنصر.
### MasonrySize: المساحة المغطاة بالطوب، مقاسة بوحدات مربعة.
### ExteriorQuality: جودة المواد المستخدمة في تغطية الجزء الخارجي.
### ExteriorCondition: الحالة الحالية للكسوة الخارجية.
### FoundationType: نوع الأساس الذي يدعم هيكل العنصر.

# الميزات الداخلية

### BasementHeight: تصنيف ارتفاع القبو بالنسبة لمستوى الأرض.
### BasementCondition: الحالة العامة لمساحة القبو.
### BasementAccess: يصف ما إذا كان للقبو وصول مشاة أو عند مستوى الحديقة.
### BasementFinish1: جودة التشطيب في المنطقة الرئيسية المنتهية من القبو.
### BasementFinishedArea1: مساحة المنطقة الرئيسية المنتهية في القبو، مقاسة بوحدات مربعة.
### BasementFinish2: جودة التشطيب في منطقة ثانية منتهية في القبو (إن وجدت).
### BasementFinishedArea2: مساحة منطقة ثانية منتهية في القبو، مقاسة بوحدات مربعة.
### BasementUnfinishedArea: مساحة الجزء غير المنتهي من القبو، مقاسة بوحدات مربعة.
### TotalBasementArea: المساحة الإجمالية للقبو، بما في ذلك المناطق المنتهية وغير المنتهية، مقاسة بوحدات مربعة.
### HeatingType: النظام المستخدم لتدفئة العنصر.
### HeatingQuality: جودة وحالة نظام التدفئة.
### AirConditioning: يشير إلى ما إذا كان العنصر يحتوي على تكييف مركزي.
### ElectricalSystem: نوع النظام الكهربائي المثبت.
### GroundFloorArea: مساحة الطابق الأرضي، مقاسة بوحدات مربعة.
### UpperFloorArea: مساحة الطابق العلوي(الطوابق)، مقاسة بوحدات مربعة.
### LowQualityArea: مساحة العنصر المنتهية بمعايير أقل، مقاسة بوحدات مربعة.
### LivingArea: المساحة الإجمالية للمعيشة فوق مستوى الأرض، مقاسة بوحدات مربعة.
### BasementFullBathrooms: عدد الحمامات الكاملة في القبو.
### BasementHalfBathrooms: عدد الحمامات النصفية في القبو.
### FullBathrooms: عدد الحمامات الكاملة فوق مستوى الأرض.
### HalfBathrooms: عدد الحمامات النصفية فوق مستوى الأرض.
### Bedrooms: عدد غرف النوم في العنصر.
### Kitchens: عدد المطابخ في العنصر.
### KitchenQuality: جودة تشطيبات وتركيبات المطبخ.
### TotalRooms: العدد الإجمالي للغرف فوق مستوى الأرض، باستثناء الحمامات.
### FunctionalityRating: تصنيف الوظائف العامة وتخطيط العنصر.
### FireplaceCount: عدد المواقد في العنصر.
### FireplaceQuality: جودة الموقد (المواقد).

# الميزات الإضافية

### GarageLocation: موقع المرآب بالنسبة للعنصر.
### GarageConstructionYear: السنة التي بُني فيها المرآب.
### GarageInterior: التشطيب الداخلي للمرآب.
### GarageCapacity: عدد المركبات التي يمكن أن يستوعبها المرآب.
### GarageSize: مساحة المرآب، مقاسة بوحدات مربعة.
### GarageQuality: جودة بناء المرآب.
### GarageCondition: الحالة الحالية للمرآب.
### DrivewayType: يشير إلى ما إذا كان الممر مرصوفًا.
### WoodDeckArea: مساحة سطح الخشب، مقاسة بوحدات مربعة.
### OpenPorchArea: مساحة الشرفة المفتوحة، مقاسة بوحدات مربعة.
### EnclosedPorchArea: مساحة الشرفة المغلقة، مقاسة بوحدات مربعة.
### ThreeSeasonPorchArea: مساحة الشرفة التي تستخدم لثلاثة فصول، مقاسة بوحدات مربعة.
### ScreenPorchArea: مساحة الشرفة المسورة، مقاسة بوحدات مربعة.
### PoolSize: مساحة المسبح، مقاسة بوحدات مربعة.
### PoolQuality: جودة المسبح.
### FenceQuality: جودة السور.
### AdditionalFeature: ميزة إضافية غير مغطاة في الفئات الأخرى.
### AdditionalFeatureValue: القيمة المالية للميزة الإضافية.

# معلومات البيع

### SaleMonth: الشهر الذي تم فيه بيع العنصر.
### SaleYear: السنة التي تم فيها بيع العنصر.
### SaleType: طريقة أو نوع عملية البيع.
### SaleCondition: حالة البيع.

In [2]:
train = pd.read_csv('train.csv')

In [ ]:
train

In [6]:
q=train.isnull().sum()
q

Id                        0
BuildingCategory          0
ZoningClassification      0
StreetLineLength        259
ParcelSize                0
                       ... 
SaleMonth                 0
SaleYear                  0
SaleType                  0
SaleCondition             0
OutcomeVariable           0
Length: 81, dtype: int64

In [7]:
q[q!=0]

StreetLineLength           259
AlleyAccessType           1369
MasonryType                872
MasonrySize                  8
BasementHeight              37
BasementCondition           37
BasementAccess              38
BasementFinish1             37
BasementFinish2             38
ElectricalSystem             1
FireplaceQuality           690
GarageLocation              81
GarageConstructionYear      81
GarageInterior              81
GarageQuality               81
GarageCondition             81
PoolQuality               1453
FenceQuality              1179
AdditionalFeature         1406
dtype: int64

In [10]:
# قراءة البيانات
data = pd.read_csv('train.csv')

# فصل البيانات إلى ميزات ومتغير الهدف
X = data.drop('OutcomeVariable', axis=1)
y = data['OutcomeVariable']

# تحديد الميزات العددية والكيفية
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# إنشاء المحول للميزات العددية والكيفية مع معالجة القيم الفارغة
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='None')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# إنشاء بايبلاين يتضمن المعالجة الأولية والنموذج
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor())
])

# ضبط الهايبر باراميترز باستخدام GridSearchCV
param_grid = {
    'regressor__n_estimators': [100, 200],
    'regressor__max_depth': [None, 10, 20],
    'regressor__min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1, scoring='r2')



R2 Score: 0.8842280142170962
Final pipeline steps: Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['Id', 'BuildingCategory', 'StreetLineLength', 'ParcelSize',
       'MaterialQuality', 'ConditionRating', 'ConstructionYear',
       'RenovationYear', 'MasonrySize', 'BasementFinishedArea1',
       'BasementFinish...
       'BasementFinish1', 'BasementFinish2', 'HeatingType', 'HeatingQuality',
       'AirConditioning', 'ElectricalSystem', 'KitchenQuality',
       'FunctionalityRating', 'FireplaceQuality', 'GarageLocation',
       'GarageInterior', 'Garag

In [ ]:
# grid_search.fit(X_train, y_train)

# # التنبؤ على مجموعة الاختبار
# y_pred = grid_search.predict(X_test)

# # حساب R-score
# r2 = r2_score(y_test, y_pred)
# print(f'R2 Score: {r2}')

# # حفظ النموذج المدرب
# joblib.dump(grid_search.best_estimator_, 'best_model.pkl')

# # وصف البايبلاين النهائي
# final_pipeline = grid_search.best_estimator_
# print(f"Final pipeline steps: {final_pipeline}")